In [101]:
import pandas as pd
import string

## Load and clean data from Rayyan

In [102]:
df = pd.read_csv("data/articles.csv")
df.head()

,key,title,year,month,day,journal,issn,volume,issue,pages,...,url,language,publisher,location,abstract,notes,doi,keywords,pubmed_id,pmc_id
0,rayyan-354359269,An experience of automated assessment in a lar...,2021.0,1,1,NaN,NaN,29.0,5.0,NaN,...,NaN,NaN,NaN,NaN,The 2020 pandemic imposed new demands on teach...,"RAYYAN-LABELS: 2021,CAE,vpl,mctest,moodle,pyth...",NaN,NaN,NaN,NaN
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,2021.0,1,1,NaN,NaN,29.0,5.0,NaN,...,NaN,NaN,NaN,NaN,Continuous assessment is essential in educatio...,"RAYYAN-LABELS: 2021,CAE,java,javassess,test ca...",NaN,NaN,NaN,NaN
2,rayyan-354359271,A tool for evaluating computer programs from s...,2021.0,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Computer science studies are more and more pop...,"RAYYAN-LABELS: 2021,EASEAI,pylint,python,metri...",NaN,Software,NaN,NaN
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,2021.0,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,This study proposes an automated programming a...,"RAYYAN-LABELS: 2021,ICKII,tool:vta,test cases,...",NaN,NaN,NaN,NaN
4,rayyan-354359273,Analysis of an automatic grading system within...,2021.0,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Reliable and pedagogically sound automated fee...,"RAYYAN-LABELS: 2021,CEP,vpl,java,mule,test cas...",NaN,NaN,NaN,NaN


### Clean data

In [103]:
df.dropna(axis=1, inplace=True)
df.drop(columns={'month', 'day'}, inplace=True)
df.head()

,key,title,authors,abstract,notes
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,"RAYYAN-LABELS: 2021,CAE,vpl,mctest,moodle,pyth..."
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,"RAYYAN-LABELS: 2021,CAE,java,javassess,test ca..."
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,"RAYYAN-LABELS: 2021,EASEAI,pylint,python,metri..."
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,"RAYYAN-LABELS: 2021,ICKII,tool:vta,test cases,..."
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,"RAYYAN-LABELS: 2021,CEP,vpl,java,mule,test cas..."


In [104]:
df.notes = df.notes.map(lambda input_str: input_str.lstrip('RAYYAN-LABELS: '))

df['screener_labels'] = df.notes.map(lambda labels: [label for label in labels.split(',')])

df.head()

,key,title,authors,abstract,notes,screener_labels
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,"2021,CAE,vpl,mctest,moodle,python,tool:vpl,dat...","[2021, CAE, vpl, mctest, moodle, python, tool:..."
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,"2021,CAE,java,javassess,test cases,correctness...","[2021, CAE, java, javassess, test cases, corre..."
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,"2021,EASEAI,pylint,python,metrics,gamified,dat...","[2021, EASEAI, pylint, python, metrics, gamifi..."
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,"2021,ICKII,tool:vta,test cases,correctness,sem...","[2021, ICKII, tool:vta, test cases, correctnes..."
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,"2021,CEP,vpl,java,mule,test cases,correctness,...","[2021, CEP, vpl, java, mule, test cases, corre..."


#### Prepend screener 1's labels with topics

In [105]:
def convert_labels(labels):
    output = []
    not_grouped = []

    for label in labels:
        if label in ['vpl', 'mctest', 'moodle', 'javassess', 'pylint', 'coderunner', 'checkstyle', 'sarfgen', 'appgrader', 'better code hub', 'matlab grader',
                     'stepik', 'earsketch', 'pmd', 'pyassessment', 'mule', 'uncode', 'jdbc', 'pass', 'cpplint', 'web-cat', 'cppcheck', 'bluej', 'submitty',
                     'juypter', 'webta', 'docker', 'pyta', 'progedu', 'siette', '2tsw', 'hipos', 'Prutor', 'atelier']:
            label = 'tool:' + label
        elif label in ['java', 'python', 'opengl', 'C', 'unknown language', 'sql', 'glsl', 'c++', 'android', 'javascript', 'c#', 'lex', 'yacc', 'Haskell', 'octave',
                       'vb', 'scala', 'matlab', 'ocaml', 'ruby', 'scheme', 'web languages', 'assesmbly', 'typescript', 'prolog', 'language  agnostic']:
            label = 'language:' + label
        elif label in ['correctness', 'code quality', 'oop', 'maintainability', 'graphics', 'complexity', 'documentation', 'concurrency',
                       'teaching context free grammar', 'best practices', 'refactoring', 'distrbuted', 'code design', 'test suite quality', 'productivity']:
            label = 'skill:' + label
        elif label in ['experience report', 'demo paper', 'evaluation paper']:
            label = 'type:' + label
        elif label in ['pre-defined questions', 'test cases', 'pattern matching', 'model solution req', 'dsl for exercise gen', 'mutants',
                       'code repair for feedback', 'metrics', 'program repair', 'static analysis', 'control flow graph', 'model-based', 'unit test expansion',
                       'dynamic analysis', 'ml', 'error prediction', 'fault localisation', 'ci', 'cluster', 'data-driven', 'matching rules', 'language stuctures', 'ci/cd', 'test case generation',
                       'token pattern matching', 'knowledge base', 'reflection', 'hipos', 'graphical output', 'nlp', 'dsl', 'antipatterns', 'mutation testing', 'uml', 'program synthesis']:
            label = 'technique:' + label
        elif label in ['internal exams', 'internal assignments', 'microsoft-dev204.1x', 'internal tutorial', 'leetcode', 'code defenders dataset', 'edX',
                       'github dataset', 'bespoke assignment', 'hackerrank', 'codechef', 'grok', 'dataset available']:
            label = 'data:' + label
        elif label in ['dataset available']:
            label = 'data_available:' + 'True'
        elif label in ['semi-automatic']:
            label = 'approach:' + label
        elif label in ['personalised feedback', 'gamified', 'pre-defined feedback', 'plagiarism', 'hints', 'nudge theory', 'visual feedback', 'manual personalised feedback', 'code repair for feedback']:
            label = 'feedback:' + label
        elif label in ['compared to human', 'compared to other tools']:
            label = 'evaluation:' + label
        else:
            not_grouped.append(label)

        output.append(label)

    print(sorted(list(set(not_grouped))))
    return output

#### Check for outstanding labels to classify

In [106]:
df.screener_labels = df.screener_labels.apply(lambda labels: convert_labels(labels))

temp_df = df.screener_labels.map(lambda labels: [label for label in labels if ':' not in label and not label.isdigit() and not label.isupper()])
temp_df[temp_df.str.len() > 0]

['2021', 'CAE', 'approach:fully automated', 'comparison:with-without', 'data:none', 'data_available:False', 'feedback:test output', 'interaction:unknown', 'tool:vpl', 'type:description', 'type:evaluation']
['2021', 'CAE', 'data_available:on request', 'evaluation:auto-grading', 'feedback:rule-failure', 'interaction:unknown', 'skill:correctness', 'skill:maintainability', 'technique:dsl-rules', 'type:description', 'type:evaluation']
['2021', 'EASEAI', 'approach:fully automated', 'data:none', 'data_available:False', 'evaluation:none', 'interaction:unknown', 'skill:maintainability', 'skill:readability', 'technique:code-metrics', 'technique:program-repair', 'technique:static-analysis', 'type:description']
['2021', 'ICKII', 'data_available:False', 'evaluation:auto-grading', 'feedback:test output', 'interaction:multiple', 'skill:correctness', 'technique:output-matching', 'tool:vta', 'type:description', 'type:evaluation']
['2021', 'CEP', 'approach:fully automated', 'data_available:False', 'eval

5                                           [ICoDSE]
13                                              [oj]
15                                    [TCAS dataset]
20                          [lms, adaptive learning]
24                                         [JITEipp]
25                                              [oj]
30                                         [SATToSE]
39                                        [accuracy]
43                                          [ICoDSE]
44                                          [EduHPC]
48     [item response theory, classical test theory]
52                                           [BotSE]
55                                          [ITiSCE]
56                                [lms, mood design]
58                                              [oj]
59                                          [ITiSCE]
61                                     [AIMLSystems]
62                                       [networked]
63                                      [oj, I

In [107]:
df.head()

,key,title,authors,abstract,notes,screener_labels
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,"2021,CAE,vpl,mctest,moodle,python,tool:vpl,dat...","[2021, CAE, tool:vpl, tool:mctest, tool:moodle..."
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,"2021,CAE,java,javassess,test cases,correctness...","[2021, CAE, language:java, tool:javassess, tec..."
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,"2021,EASEAI,pylint,python,metrics,gamified,dat...","[2021, EASEAI, tool:pylint, language:python, t..."
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,"2021,ICKII,tool:vta,test cases,correctness,sem...","[2021, ICKII, tool:vta, technique:test cases, ..."
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,"2021,CEP,vpl,java,mule,test cases,correctness,...","[2021, CEP, tool:vpl, language:java, tool:mule..."


### Separate out year published

In [108]:
# Get year from list adapted from: https://stackoverflow.com/questions/363944/python-idiom-to-return-first-item-or-none

df['year_published'] = df.screener_labels.apply(lambda labels: next(iter([label for label in labels if label.isdigit()] or []), None))
df.year_published = df.year_published.fillna(0)
df.year_published = df.year_published.astype('int')
df.screener_labels = df.screener_labels.apply(lambda  labels: [label for label in labels if not label.isdigit()])

df.screener_labels = df.screener_labels.map(lambda labels: [label for label in labels if ":" in label])
df.screener_labels = df.screener_labels.apply(lambda labels: [label.replace('comparison', 'evaluation') for label in labels])

df.head()

,key,title,authors,abstract,notes,screener_labels,year_published
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,"2021,CAE,vpl,mctest,moodle,python,tool:vpl,dat...","[tool:vpl, tool:mctest, tool:moodle, language:...",2021
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,"2021,CAE,java,javassess,test cases,correctness...","[language:java, tool:javassess, technique:test...",2021
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,"2021,EASEAI,pylint,python,metrics,gamified,dat...","[tool:pylint, language:python, technique:metri...",2021
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,"2021,ICKII,tool:vta,test cases,correctness,sem...","[tool:vta, technique:test cases, skill:correct...",2021
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,"2021,CEP,vpl,java,mule,test cases,correctness,...","[tool:vpl, language:java, tool:mule, technique...",2021


In [109]:
df[df.year_published == 2016]

,key,title,authors,abstract,notes,screener_labels,year_published


#### Clean tags

In [110]:
punct = string.punctuation.replace(':', '').replace('#', '').replace('+', '') + " "

In [111]:
def clean_tags(labels):
    return [label.translate(str.maketrans(dict.fromkeys(punct, "_"))) for label in labels]

In [112]:
df.screener_labels = df.screener_labels.apply(lambda labels: clean_tags(labels))

df.head()

,key,title,authors,abstract,notes,screener_labels,year_published
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,"2021,CAE,vpl,mctest,moodle,python,tool:vpl,dat...","[tool:vpl, tool:mctest, tool:moodle, language:...",2021
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,"2021,CAE,java,javassess,test cases,correctness...","[language:java, tool:javassess, technique:test...",2021
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,"2021,EASEAI,pylint,python,metrics,gamified,dat...","[tool:pylint, language:python, technique:metri...",2021
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,"2021,ICKII,tool:vta,test cases,correctness,sem...","[tool:vta, technique:test_cases, skill:correct...",2021
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,"2021,CEP,vpl,java,mule,test cases,correctness,...","[tool:vpl, language:java, tool:mule, technique...",2021


In [113]:
df['tags'] = df.screener_labels
df.tags = df.tags.apply(lambda tags: list(sorted(set(tags))))

df = df.drop(columns=['screener_labels', 'notes'])

df.head()

,key,title,authors,abstract,year_published,tags
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,2021,"[approach:fully_automated, data:none, data_ava..."
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,2021,"[approach:semi_automatic, data:internal_exams,..."
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,2021,"[approach:fully_automated, data:none, data_ava..."
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,2021,"[approach:semi_automatic, data:internal_assign..."
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,2021,"[approach:fully_automated, data:internal_assig..."


## Validate data

#### Get all unique tags

In [114]:
sorted(set([tag for list_of_tags in df.tags for tag in list_of_tags]))

['approach:fully_automated',
 'approach:semi_automatic',
 'approach:unclear',
 'data:bespoke_assignment',
 'data:code_defenders_dataset',
 'data:codechef',
 'data:dataset_available',
 'data:edX',
 'data:github_dataset',
 'data:grok',
 'data:hackerrank',
 'data:internal_assignments',
 'data:internal_exams',
 'data:internal_tutorial',
 'data:leetcode',
 'data:microsoft_dev204_1x',
 'data:none',
 'data:text_book_exercises',
 'data_available:False',
 'data_available:True',
 'data_available:on_request',
 'evaluation:analytics',
 'evaluation:anecdotal',
 'evaluation:auto_graders',
 'evaluation:auto_grading',
 'evaluation:compared_to_human',
 'evaluation:compared_to_other_tools',
 'evaluation:correlation_grades',
 'evaluation:engagement',
 'evaluation:feedback',
 'evaluation:grade_correlation',
 'evaluation:grading_accuracy',
 'evaluation:manual_grading',
 'evaluation:ml_models',
 'evaluation:none',
 'evaluation:student_survey',
 'evaluation:teacher_survey',
 'evaluation:usage',
 'evaluation:

#### Rename duplicate tags

In [115]:
def rename_duplicate_tags(labels):
    new_labels = []
    for label in labels:
        if label == 'type:evaluation_paper':
            label = 'type:evaluation'
        elif label == 'technique:test_cases':
            label = 'technique:unit_testing'
        elif label == 'technique:metrics':
            label = 'technique:code_metrics'
        elif label == 'technique:ml':
            label = 'technique:machine_learning'
        elif label == 'evaluation:auto_graders':
            label = 'evaluation:auto_grading'
        elif label in ['evaluation:with_without', 'evaluation:compared_to_human']:
            label = 'evaluation:manual_grading'
        elif label == 'technique:ci':
            label = 'technique:ci_cd'
        elif label == 'technique:cluster':
            label = 'technique:clustering'
        elif label == 'technique:mutants':
            label = 'technique:mutation_testing'
        elif label in ['technique:dsl', 'technique:dsl_rules', 'technique:rule_based']:
            label = 'technique:matching_rules'
        elif label == 'language:assesmbly':
            label = 'language:assembly'
        elif label == 'language:kernal':
            label = 'language:kernel'
        elif label == 'skill:distrbuted':
            label = 'skill:distributed'

        new_labels.append(label)

    return list(set(new_labels))


In [116]:
df.tags = df.tags.apply(lambda labels: rename_duplicate_tags(labels))
sorted(set([tag for list_of_tags in df.tags for tag in list_of_tags]))

['approach:fully_automated',
 'approach:semi_automatic',
 'approach:unclear',
 'data:bespoke_assignment',
 'data:code_defenders_dataset',
 'data:codechef',
 'data:dataset_available',
 'data:edX',
 'data:github_dataset',
 'data:grok',
 'data:hackerrank',
 'data:internal_assignments',
 'data:internal_exams',
 'data:internal_tutorial',
 'data:leetcode',
 'data:microsoft_dev204_1x',
 'data:none',
 'data:text_book_exercises',
 'data_available:False',
 'data_available:True',
 'data_available:on_request',
 'evaluation:analytics',
 'evaluation:anecdotal',
 'evaluation:auto_grading',
 'evaluation:compared_to_other_tools',
 'evaluation:correlation_grades',
 'evaluation:engagement',
 'evaluation:feedback',
 'evaluation:grade_correlation',
 'evaluation:grading_accuracy',
 'evaluation:manual_grading',
 'evaluation:ml_models',
 'evaluation:none',
 'evaluation:student_survey',
 'evaluation:teacher_survey',
 'evaluation:usage',
 'evaluation:user_study',
 'feedback:augmented_compiler_errors',
 'feedbac

In [117]:
required_prefixes = {'approach', 'language', 'type', 'skill', 'technique', 'data', 'evaluation', 'feedback', 'tool', 'data_available', 'interaction'}

In [118]:
def validate(labels):
    paper_prefixes = []
    for label in labels:
        paper_prefixes.append(label.split(':')[0])

    return sorted([prefix for prefix in required_prefixes if prefix not in paper_prefixes])

validate_df = pd.concat([df.title, df.tags.apply(lambda labels: validate(labels))], axis=1)
validate_df.to_csv('data/validate.csv')

### Add categories column

In [119]:
# Reread paper:
# A Comparison of Inquiry-Based Conceptual Feedback vs. Traditional Detailed Feedback Mechanisms in Software Testing Education: An Empirical Investigation

def categorise(labels):
    categories = []
    if {'technique:unit_testing', 'technique:property_based_testing'}.intersection(labels):
        categories.append('dynamic')
    if {'technique:pattern_matching', 'technique:static_analysis', 'technique:code_metrics', 'technique:clustering',
        'technique:style_check', 'technique:program_repair', 'technique:matching_rules'}.intersection(labels):
        categories.append('static')
    if 'technique:machine_learning' in labels:
        categories.append('machine learning')

    if len(categories) == 0:
        return None

    return categories

In [120]:
df['category'] = df.tags.apply(lambda labels: categorise(labels))
pd.concat([df.title, df.tags.apply(lambda labels: set([label for label in labels if 'technique' in label])), df.category], axis=1)

,title,tags,category
0,An experience of automated assessment in a lar...,"{technique:pre_defined_questions, technique:un...",[dynamic]
1,Semiautomatic generation and assessment of Jav...,"{technique:code_repair_for_feedback, technique...","[dynamic, static]"
2,A tool for evaluating computer programs from s...,"{technique:code_metrics, technique:static_anal...",[static]
3,Virtual Teaching Assistant for Grading Program...,"{technique:output_matching, technique:unit_tes...","[dynamic, static]"
4,Analysis of an automatic grading system within...,"{technique:matching_rules, technique:unit_test...","[dynamic, static]"
...,...,...,...
117,Automated data-driven hints for computer progr...,{technique:unit_testing},[dynamic]
118,Learning and teaching numerical methods with a...,"{technique:unit_testing, technique:matching_ru...","[dynamic, static]"
119,Ask-Elle: an Adaptable Programming Tutor for H...,"{technique:property_based_testing, technique:m...",[dynamic]
120,An AI system for coaching novice programmers,"{technique:pre_defined_questions, technique:co...","[static, machine learning]"


### Add Skills Column

In [121]:
def add_skills(labels):
    skills = []

    if 'skill:correctness' in labels:
        skills.append('correctness')
    if {'skill:code_quality', 'skill:readability', 'skill:refactoring'}.intersection(labels):
        skills.append('readability')
    if {'skill:maintainability', 'skill:refactoring', 'skill:test_suite_quality', 'skill:code_design',
            'skill:complexity', 'skill:oop'}.intersection(labels):
        skills.append('maintainability')
    if 'skill:documentation' in labels:
        skills.append('documentation')


    if len(skills) == 0:
        return None

    return skills

In [122]:
df['skills'] = df.tags.apply(lambda labels: add_skills(labels))
pd.concat([df.title, df.tags.apply(lambda labels: set([label for label in labels if 'skill' in label])), df.skills], axis=1)

,title,tags,skills
0,An experience of automated assessment in a lar...,{skill:correctness},[correctness]
1,Semiautomatic generation and assessment of Jav...,"{skill:correctness, skill:maintainability}","[correctness, maintainability]"
2,A tool for evaluating computer programs from s...,"{skill:code_quality, skill:maintainability, sk...","[readability, maintainability]"
3,Virtual Teaching Assistant for Grading Program...,{skill:correctness},[correctness]
4,Analysis of an automatic grading system within...,"{skill:code_quality, skill:correctness}","[correctness, readability]"
...,...,...,...
117,Automated data-driven hints for computer progr...,{skill:correctness},[correctness]
118,Learning and teaching numerical methods with a...,{skill:correctness},[correctness]
119,Ask-Elle: an Adaptable Programming Tutor for H...,"{skill:code_design, skill:correctness}","[correctness, maintainability]"
120,An AI system for coaching novice programmers,"{skill:code_quality, skill:maintainability}","[readability, maintainability]"


### Add Types Column

In [123]:
def add_types(labels):
    types = []

    if 'type:description' in labels:
        types.append('description')
    if 'type:evaluation' in labels:
        types.append('evaluation')
    if 'type:experience_report' in labels:
        types.append('experience')
    if 'type:demo_paper' in labels:
        types.append('demo')

    return types

In [124]:
df['types'] = df.tags.apply(lambda labels: add_types(labels))
pd.concat([df.title, df.tags.apply(lambda labels: set([label for label in labels if 'type' in label])), df.types], axis=1)

,title,tags,types
0,An experience of automated assessment in a lar...,"{type:description, type:evaluation}","[description, evaluation]"
1,Semiautomatic generation and assessment of Jav...,"{type:description, type:evaluation}","[description, evaluation]"
2,A tool for evaluating computer programs from s...,{type:description},[description]
3,Virtual Teaching Assistant for Grading Program...,"{type:description, type:evaluation}","[description, evaluation]"
4,Analysis of an automatic grading system within...,"{type:description, type:evaluation}","[description, evaluation]"
...,...,...,...
117,Automated data-driven hints for computer progr...,"{type:description, type:evaluation}","[description, evaluation]"
118,Learning and teaching numerical methods with a...,{type:description},[description]
119,Ask-Elle: an Adaptable Programming Tutor for H...,"{type:description, type:evaluation}","[description, evaluation]"
120,An AI system for coaching novice programmers,"{type:description, type:evaluation}","[description, evaluation]"


### Add language family column

In [125]:
temp = df.tags.apply(lambda labels: str(set([label for label in labels if 'language:' in label])))
set(temp.tolist())

{"{'language:C', 'language:Haskell'}",
 "{'language:C', 'language:c++'}",
 "{'language:C', 'language:python', 'language:language__agnostic'}",
 "{'language:C', 'language:python'}",
 "{'language:C'}",
 "{'language:Haskell'}",
 "{'language:android'}",
 "{'language:assembly'}",
 "{'language:c#'}",
 "{'language:c++', 'language:glsl'}",
 "{'language:c++'}",
 "{'language:glsl'}",
 "{'language:java', 'language:C', 'language:python', 'language:c++'}",
 "{'language:java', 'language:python'}",
 "{'language:java', 'language:scala'}",
 "{'language:java'}",
 "{'language:kernel'}",
 "{'language:language__agnostic'}",
 "{'language:lex', 'language:C', 'language:yacc', 'language:language__agnostic'}",
 "{'language:matlab', 'language:C'}",
 "{'language:matlab'}",
 "{'language:ocaml'}",
 "{'language:opengl', 'language:c++'}",
 "{'language:opengl'}",
 "{'language:prolog'}",
 "{'language:python', 'language:c++'}",
 "{'language:python', 'language:javascript'}",
 "{'language:python', 'language:octave'}",
 "{

In [126]:
# Are these correct?
# Check papers: 48, 59
def add_lang_family(labels):
    family = []

    if {'language:java', 'language:python', 'language:C', 'language:matlab', 'language:octave',
        'language:c++', 'language:android', 'language:vb', 'language:c#', 'language:ruby'}.intersection(labels):
        family.append('OOP')
    if {'language:Haskell', 'language:scala', 'language:ocaml', 'language:scheme'}.intersection(labels):
        family.append('Functional')
    if 'language:prolog' in labels:
        family.append('Logic')
    if {'language:javascript', 'language:typescript', 'language:web_languages'}.intersection(labels):
        family.append('Web')
    if {'language:kernel', 'language:assembly'}.intersection(labels):
        family.append('Low-Level')
    if {'language:opengl', 'language:glsl'}.intersection(labels):
        family.append('Graphics')
    if 'language:sql' in labels:
        family.append('Query')
    if 'language:unknown_language' in labels:
        family.append('Unknown')
    if 'language:language__agnostic' in labels:
        family.append('Agnostic')

    if len(family) == 0:
        return None

    return family

In [127]:
df['lang_family'] = df.tags.apply(lambda labels: add_lang_family(labels))
pd.concat([df.title, df.tags.apply(lambda labels: set([label for label in labels if 'language:' in label])), df.lang_family], axis=1)

,title,tags,lang_family
0,An experience of automated assessment in a lar...,{language:python},[OOP]
1,Semiautomatic generation and assessment of Jav...,{language:java},[OOP]
2,A tool for evaluating computer programs from s...,{language:python},[OOP]
3,Virtual Teaching Assistant for Grading Program...,{language:unknown_language},[Unknown]
4,Analysis of an automatic grading system within...,{language:java},[OOP]
...,...,...,...
117,Automated data-driven hints for computer progr...,{language:python},[OOP]
118,Learning and teaching numerical methods with a...,"{language:python, language:octave}",[OOP]
119,Ask-Elle: an Adaptable Programming Tutor for H...,{language:Haskell},[Functional]
120,An AI system for coaching novice programmers,{language:java},[OOP]


#### Check no NA

In [128]:
df.isna().sum()

key               0
title             0
authors           0
abstract          0
year_published    0
tags              0
category          1
skills            0
types             0
lang_family       0
dtype: int64

## Export

In [129]:
df.to_csv('data/processed.csv')

# Example analytics count of all tags

In [130]:
pd.Series([tag for tags in df.tags for tag in tags]).value_counts()

skill:correctness                   111
data_available:False                104
approach:fully_automated            100
type:evaluation                      99
type:description                     94
                                   ... 
technique:novel                       1
skill:concurrency                     1
tool:dbms                             1
technique:token_pattern_matching      1
tool:codq                             1
Length: 259, dtype: int64

In [131]:
df.dtypes

key               object
title             object
authors           object
abstract          object
year_published     int64
tags              object
category          object
skills            object
types             object
lang_family       object
dtype: object